In [1]:
import xgboost as xgb
import pandas as pd

# Reading and Joining Datasets 

In [2]:
train = pd.read_csv('train_store.csv', parse_dates=['Date'])
store = pd.read_csv('store.csv')
test = pd.read_csv('test.csv', parse_dates=['Date'])

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1.0,5,2015-07-31,5263.0,555.0,1.0,1.0,0,1.0
1,2.0,5,2015-07-31,6064.0,625.0,1.0,1.0,0,1.0
2,3.0,5,2015-07-31,8314.0,821.0,1.0,1.0,0,1.0
3,4.0,5,2015-07-31,13995.0,1498.0,1.0,1.0,0,1.0
4,5.0,5,2015-07-31,4822.0,559.0,1.0,1.0,0,1.0


In [4]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [6]:
train = train.merge(store, on='Store', copy=False)

# Processing Train Dataset

In [7]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1.0,5,2015-07-31,5263.0,555.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1.0,4,2015-07-30,5020.0,546.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1.0,3,2015-07-29,4782.0,523.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1.0,2,2015-07-28,5011.0,560.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1.0,1,2015-07-27,6102.0,612.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


# Converting Date column into particular shape

In [12]:
train['year'] = train.Date.apply(lambda x: x.year)
train['month'] = train.Date.apply(lambda x: x.month)
train['woy'] = train.Date.apply(lambda x: x.weekofyear)
del train['Date']
train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,woy
0,1.0,5,5263.0,555.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31
1,1.0,4,5020.0,546.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31
2,1.0,3,4782.0,523.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31
3,1.0,2,5011.0,560.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31
4,1.0,1,6102.0,612.0,1.0,1.0,0,1.0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31


# Competition Open time in months

In [16]:
train['CompetitionOpenInMonths'] = (12 - train.CompetitionOpenSinceMonth) + (train.month) + 12*(train.year - train.CompetitionOpenSinceYear-1)
train['CompetitionOpenInMonths'] = train.CompetitionOpenInMonths.apply(lambda x: x if x > 0 else 0)
del train['CompetitionOpenSinceMonth']
del train['CompetitionOpenSinceYear']
train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,woy,CompetitionOpenInMonths
0,1.0,5,5263.0,555.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,NaN,NaN,2015,7,31,82.0
1,1.0,4,5020.0,546.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,NaN,NaN,2015,7,31,82.0
2,1.0,3,4782.0,523.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,NaN,NaN,2015,7,31,82.0
3,1.0,2,5011.0,560.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,NaN,NaN,2015,7,31,82.0
4,1.0,1,6102.0,612.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,NaN,NaN,2015,7,31,82.0


# Promo2 Open time in motnhs

In [19]:
train['PromoOpenInMonths'] = (48 - train.Promo2SinceWeek)/4.0 + train.woy/4.0 + 12*(train.year - train.Promo2SinceYear - 1)
train['PromoOpenInMonths'] = train.PromoOpenInMonths.apply(lambda x: x if x > 0 else 0)
del train['Promo2SinceYear']
del train['Promo2SinceWeek']
train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,year,month,woy,CompetitionOpenInMonths,PromoOpenInMonths
0,1.0,5,5263.0,555.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,2015,7,31,82.0,0.0
1,1.0,4,5020.0,546.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,2015,7,31,82.0,0.0
2,1.0,3,4782.0,523.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,2015,7,31,82.0,0.0
3,1.0,2,5011.0,560.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,2015,7,31,82.0,0.0
4,1.0,1,6102.0,612.0,1.0,1.0,0,1.0,c,a,1270.0,0,NaN,2015,7,31,82.0,0.0


# Converting PromoInterval into particular shape

In [29]:
train['month1'] = train.PromoInterval.apply(lambda x: x.split(',')[0] if type(x) == str else 0)
train['month2'] = train.PromoInterval.apply(lambda x: x.split(',')[1] if type(x) == str else 0)
train['month3'] = train.PromoInterval.apply(lambda x: x.split(',')[2] if type(x) == str else 0)
train['month4'] = train.PromoInterval.apply(lambda x: x.split(',')[3] if type(x) == str else 0)
del train['PromoInterval'] 
train.tail(10)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,year,month,woy,CompetitionOpenInMonths,PromoOpenInMonths,month1,month2,month3,month4
1017198,1115.0,4,5007.0,339.0,1.0,1.0,0,1.0,d,c,...,1,2013,1,2,0.0,7.00,Mar,Jun,Sept,Dec
1017199,1115.0,3,4649.0,324.0,1.0,1.0,0,1.0,d,c,...,1,2013,1,2,0.0,7.00,Mar,Jun,Sept,Dec
1017200,1115.0,2,5243.0,341.0,1.0,1.0,0,1.0,d,c,...,1,2013,1,2,0.0,7.00,Mar,Jun,Sept,Dec
1017201,1115.0,1,6905.0,471.0,1.0,1.0,0,1.0,d,c,...,1,2013,1,2,0.0,7.00,Mar,Jun,Sept,Dec
1017202,1115.0,7,0.0,0.0,0.0,0.0,0,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec
1017203,1115.0,6,4771.0,339.0,1.0,0.0,0,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec
1017204,1115.0,5,4540.0,326.0,1.0,0.0,0,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec
1017205,1115.0,4,4297.0,300.0,1.0,0.0,0,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec
1017206,1115.0,3,3697.0,305.0,1.0,0.0,0,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec
1017207,1115.0,2,0.0,0.0,0.0,0.0,a,1.0,d,c,...,1,2013,1,1,0.0,6.75,Mar,Jun,Sept,Dec


# Filling NaN values

In [34]:
train.isnull().any()

Store                      False
DayOfWeek                  False
Sales                      False
Customers                  False
Open                       False
Promo                      False
StateHoliday               False
SchoolHoliday              False
StoreType                  False
Assortment                 False
CompetitionDistance         True
Promo2                     False
year                       False
month                      False
woy                        False
CompetitionOpenInMonths    False
PromoOpenInMonths          False
month1                     False
month2                     False
month3                     False
month4                     False
dtype: bool

In [37]:
train.CompetitionDistance.fillna(0, inplace=True)

# Converting Categorical values to Numerical values

In [36]:
train.dtypes

Store                      float64
DayOfWeek                    int64
Sales                      float64
Customers                  float64
Open                       float64
Promo                      float64
StateHoliday                object
SchoolHoliday              float64
StoreType                   object
Assortment                  object
CompetitionDistance        float64
Promo2                       int64
year                         int64
month                        int64
woy                          int64
CompetitionOpenInMonths    float64
PromoOpenInMonths          float64
month1                      object
month2                      object
month3                      object
month4                      object
dtype: object

In [39]:
train.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0L], dtype=object)

In [40]:
train['StateHoliday'] = train['StateHoliday'].replace({'0':0, 'a':1, 'b':2, 'c':3})

In [43]:
train.StateHoliday.unique()

array([0, 1, 2, 3], dtype=int64)

In [44]:
train.StoreType.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [45]:
train['StoreType'] = train['StoreType'].replace({'a':1, 'b':2, 'c':3, 'd':4})

In [46]:
train.StoreType.unique()

array([3, 1, 4, 2], dtype=int64)

In [47]:
train.Assortment.unique()

array(['a', 'c', 'b'], dtype=object)

In [48]:
train['Assortment'] = train['Assortment'].replace({'a':1, 'b':2, 'c':3})

In [49]:
train.Assortment.unique()

array([1, 3, 2], dtype=int64)

In [52]:
train = pd.get_dummies(train, columns = ['month1', 'month2', 'month3', 'month4'])                            

In [53]:
train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,month2_Jun,month2_May,month3_0,month3_Aug,month3_Jul,month3_Sept,month4_0,month4_Dec,month4_Nov,month4_Oct
0,1.0,5,5263.0,555.0,1.0,1.0,0,1.0,3,1,...,0,0,1,0,0,0,1,0,0,0
1,1.0,4,5020.0,546.0,1.0,1.0,0,1.0,3,1,...,0,0,1,0,0,0,1,0,0,0
2,1.0,3,4782.0,523.0,1.0,1.0,0,1.0,3,1,...,0,0,1,0,0,0,1,0,0,0
3,1.0,2,5011.0,560.0,1.0,1.0,0,1.0,3,1,...,0,0,1,0,0,0,1,0,0,0
4,1.0,1,6102.0,612.0,1.0,1.0,0,1.0,3,1,...,0,0,1,0,0,0,1,0,0,0


In [54]:
del train['month1_0']
del train['month2_0']
del train['month3_0']
del train['month4_0']

In [55]:
train.dtypes

Store                      float64
DayOfWeek                    int64
Sales                      float64
Customers                  float64
Open                       float64
Promo                      float64
StateHoliday                 int64
SchoolHoliday              float64
StoreType                    int64
Assortment                   int64
CompetitionDistance        float64
Promo2                       int64
year                         int64
month                        int64
woy                          int64
CompetitionOpenInMonths    float64
PromoOpenInMonths          float64
month1_Feb                   uint8
month1_Jan                   uint8
month1_Mar                   uint8
month2_Apr                   uint8
month2_Jun                   uint8
month2_May                   uint8
month3_Aug                   uint8
month3_Jul                   uint8
month3_Sept                  uint8
month4_Dec                   uint8
month4_Nov                   uint8
month4_Oct          

# Processing Test Dataset

In [4]:
test = pd.read_csv('test.csv', parse_dates=['Date'])
store = pd.read_csv('store.csv')
test = test.merge(store, on='Store', copy=False)

In [5]:
test['year'] = test.Date.apply(lambda x: x.year)
test['month'] = test.Date.apply(lambda x: x.month)
test['woy'] = test.Date.apply(lambda x: x.weekofyear)
del test['Date']

test['CompetitionOpenInMonths'] = (12 - test.CompetitionOpenSinceMonth) + (test.month) + 12*(test.year - test.CompetitionOpenSinceYear-1)
test['CompetitionOpenInMonths'] = test.CompetitionOpenInMonths.apply(lambda x: x if x > 0 else 0)
del test['CompetitionOpenSinceMonth']
del test['CompetitionOpenSinceYear']

test['PromoOpenInMonths'] = (48 - test.Promo2SinceWeek)/4.0 + test.woy/4.0 + 12*(test.year - test.Promo2SinceYear - 1)
test['PromoOpenInMonths'] = test.PromoOpenInMonths.apply(lambda x: x if x > 0 else 0)
del test['Promo2SinceYear']
del test['Promo2SinceWeek']

test['month1'] = test.PromoInterval.apply(lambda x: x.split(',')[0] if type(x) == str else 0)
test['month2'] = test.PromoInterval.apply(lambda x: x.split(',')[1] if type(x) == str else 0)
test['month3'] = test.PromoInterval.apply(lambda x: x.split(',')[2] if type(x) == str else 0)
test['month4'] = test.PromoInterval.apply(lambda x: x.split(',')[3] if type(x) == str else 0)
del test['PromoInterval'] 

test.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,year,month,woy,CompetitionOpenInMonths,PromoOpenInMonths,month1,month2,month3,month4
0,1,1,4,1.0,1,0,0,c,a,1270.0,0,2015,9,38,84.0,0.0,0,0,0,0
1,857,1,3,1.0,1,0,0,c,a,1270.0,0,2015,9,38,84.0,0.0,0,0,0,0
2,1713,1,2,1.0,1,0,0,c,a,1270.0,0,2015,9,38,84.0,0.0,0,0,0,0
3,2569,1,1,1.0,1,0,0,c,a,1270.0,0,2015,9,38,84.0,0.0,0,0,0,0
4,3425,1,7,0.0,0,0,0,c,a,1270.0,0,2015,9,37,84.0,0.0,0,0,0,0


In [6]:
test.Open.fillna(0, inplace=True)
test.CompetitionDistance.fillna(0, inplace=True)

In [7]:
test.Assortment.unique()

array(['a', 'c', 'b'], dtype=object)

In [13]:
test.StateHoliday.unique()

array(['0', 'a'], dtype=object)

In [14]:
test['StateHoliday'] = test['StateHoliday'].map({'0':0, 'a':1})

In [15]:
test.StoreType.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [16]:
test['StoreType'] = test['StoreType'].map({'a':1, 'b':2, 'c':3, 'd':4})

In [8]:
test['Assortment'] = test['Assortment'].replace({'a':1, 'b':2, 'c':3})

In [9]:
test = pd.get_dummies(test, columns = ['month1', 'month2', 'month3', 'month4']) 

In [10]:
del test['month1_0']
del test['month2_0']
del test['month3_0']
del test['month4_0']

In [85]:
train.to_csv('clean_train.csv')
test.to_csv('clean_test.csv')

# Loading cleaned train and test datasets

In [3]:
import xgboost as xgb
import pandas as pd

train = pd.read_csv('clean_train.csv')
del train['Unnamed: 0']
test = pd.read_csv('clean_test.csv')
del test['Unnamed: 0'] 
train = train[train.Open == 1]

In [4]:
x_train = train.drop(['Sales', 'Customers', 'year', 'Store'], axis = 1)
y_train = train[['Sales']]

# Tuning a training model (XGBoost Regressor)

In [5]:
from sklearn.model_selection import GridSearchCV

In [174]:
from sklearn.model_selection import GridSearchCV
cv_params = {'learning_rate': [0.01, 0.02, 0.03], 'max_depth': [4, 6, 8]}
optimal_xgb = GridSearchCV(xgb.XGBClassifier(), cv_params, cv = 3, n_jobs = -1)
optimal_xgb.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [15, 20, 25], 'learning_rate': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [175]:
optimal_xgb.grid_scores_, optimal_xgb.best_params_, optimal_xgb.best_score_

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.00286, std: 0.04862, params: {'n_estimators': 15, 'learning_rate': 0.1},
  mean: 0.14986, std: 0.03933, params: {'n_estimators': 20, 'learning_rate': 0.1},
  mean: 0.20746, std: 0.03010, params: {'n_estimators': 25, 'learning_rate': 0.1},
  mean: 0.23456, std: 0.02624, params: {'n_estimators': 15, 'learning_rate': 0.2},
  mean: 0.25044, std: 0.01816, params: {'n_estimators': 20, 'learning_rate': 0.2},
  mean: 0.25436, std: 0.01802, params: {'n_estimators': 25, 'learning_rate': 0.2},
  mean: 0.25491, std: 0.01739, params: {'n_estimators': 15, 'learning_rate': 0.3},
  mean: 0.25647, std: 0.01509, params: {'n_estimators': 20, 'learning_rate': 0.3},
  mean: 0.25668, std: 0.01405, params: {'n_estimators': 25, 'learning_rate': 0.3}],
 {'learning_rate': 0.3, 'n_estimators': 25},
 0.2566795333612239)

In [318]:
cv_params = {'max_depth': [6, 8, 10], 'min_child_weight':[6, 8, 10]}
optimal_xgb = GridSearchCV(xgb.XGBRegressor(learning_rate=0.3), cv_params, cv = 3, n_jobs = -1)
optimal_xgb.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [6, 8, 10], 'min_child_weight': [6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [319]:
optimal_xgb.grid_scores_, optimal_xgb.best_params_, optimal_xgb.best_score_

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.06801, std: 0.04376, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: 0.07050, std: 0.03605, params: {'max_depth': 6, 'min_child_weight': 8},
  mean: 0.04713, std: 0.06327, params: {'max_depth': 6, 'min_child_weight': 10},
  mean: 0.02366, std: 0.05529, params: {'max_depth': 8, 'min_child_weight': 6},
  mean: 0.02562, std: 0.05725, params: {'max_depth': 8, 'min_child_weight': 8},
  mean: 0.02524, std: 0.05572, params: {'max_depth': 8, 'min_child_weight': 10},
  mean: 0.01778, std: 0.02570, params: {'max_depth': 10, 'min_child_weight': 6},
  mean: 0.01400, std: 0.05686, params: {'max_depth': 10, 'min_child_weight': 8},
  mean: -0.00652, std: 0.01262, params: {'max_depth': 10, 'min_child_weight': 10}],
 {'max_depth': 6, 'min_child_weight': 8},
 0.070504323030700297)

In [321]:
cv_params = {'subsample':[i/10.0 for i in range(6,10)], 'colsample_bytree':[i/10.0 for i in range(6,10)]}
optimal_xgb = GridSearchCV(xgb.XGBRegressor(learning_rate=0.3, max_depth=6, min_child_weight=8), cv_params, cv = 3, n_jobs = -1)
optimal_xgb.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=6,
       min_child_weight=8, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [322]:
optimal_xgb.grid_scores_, optimal_xgb.best_params_, optimal_xgb.best_score_

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.03727, std: 0.02985, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.03196, std: 0.03556, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.06674, std: 0.03343, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.05616, std: 0.00994, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.07054, std: 0.05988, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.07687, std: 0.04340, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.00932, std: 0.05051, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.06087, std: 0.04858, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.05614, std: 0.06237, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.03479, std: 0.02944, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.04571, std: 0.06507, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.03092, std: 0.01228, params: {'subsample': 0.9, 'colsample_bytree'

# Here is the final Model!

In [6]:
model = xgb.XGBRegressor(learning_rate = 0.3, max_depth=10, min_child_weight=6, subsample=0.7, colsample_bytree=0.7).fit(x_train, y_train)
print 'Model was created and trained'

Model was created and trained


In [7]:
test1 = test[test.Open == 1]
x_test = test1.drop(['year', 'Store', 'Id'], axis=1)
test1_id = test1[['Id']]
test_id = test[['Id']]

In [8]:
y_test = model.predict(x_test)

In [9]:
test1_id['Sales'] = y_test

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
test1_id.sort_values('Id', inplace=True)

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
test1_id.set_index('Id', inplace=True)

In [12]:
test_id.sort_values('Id', inplace=True)
test_id.set_index('Id', inplace=True)

C:\Users\Sysadmin\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
result = pd.concat([test_id, test1_id], axis=1, join_axes=[test_id.index])

In [14]:
result.fillna(0, inplace=True)

In [15]:
result.head()

,Sales
Id,
1,4950.506348
2,8138.780762
3,8965.261719
4,6049.634766
5,6408.037598


In [16]:
result.to_csv('submission_during_class_xgboost_rossman.csv')
print 'submission file is ready' 

submission file is ready
